In [ ]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
root_path = "../output/human_ai_study/"
ppt_data_paths = glob.glob(f"{root_path}*")
ppt_data_paths.sort(key=lambda x: x.split("/")[-1].split("_")[-1])
ppt_ids = [x.split('/')[-1] for x in ppt_data_paths]
ppt_data_paths = list(zip(ppt_ids, ppt_data_paths))[:20]

In [ ]:
tasks = {
    "1st_session": ["pendulum", "rdk_alone_first", "rdk_alone_AI_assist_first", "rdk_alone_hitl"],
    "2nd_session": ["rdk_alone_second", "rdk_alone_AI_assist_second_original", "rdk_alone_AI_assist_second_retrained"]
}

In [ ]:
'''
returns the average subarray size where continuous inputs are non-zero
'''
def average_nonzero_subarray_size(arr):
    nonzero_subarray_sizes = []
    current_subarray_size = 0

    for value in arr:
        if value != 0:
            # Inside a contiguous subarray with non-zero values
            current_subarray_size += 1
        elif value == 0 and current_subarray_size > 0:
            # End of a contiguous subarray with non-zero values
            nonzero_subarray_sizes.append(current_subarray_size)
            current_subarray_size = 0

    if current_subarray_size > 0:
        nonzero_subarray_sizes.append(current_subarray_size)

    if nonzero_subarray_sizes:
        return sum(nonzero_subarray_sizes) / len(nonzero_subarray_sizes)
    else:
        return 0

'''
returns the average subarray size where continuous inputs are zero
'''
def average_zero_subarray_size(arr):
    interval_sizes = []
    current_interval_size = 0

    for value in arr:
        if value == 0 and current_interval_size >= 0:
            # Inside an interval
            current_interval_size += 1
        elif value != 0 and current_interval_size > 0:
            # End of an interval
            interval_sizes.append(current_interval_size)
            current_interval_size = 0

    if current_interval_size > 0:
        interval_sizes.append(current_interval_size)

    if interval_sizes:
        return sum(interval_sizes) / len(interval_sizes)
    else:
        return 0

In [ ]:
def summarize_dataframe(df, index, isCatch):
    times = np.array(df['time'])
    num_actions = np.size(df["destabilizing_actions"])
    num_destabilizing_actions = np.sum(df["destabilizing_actions"])
    destabilizing_prop = num_destabilizing_actions/num_actions
    num_crash_cond_triggers = np.sum(df["is_crash_condition_triggered"])
    crash_cond_triggered_prop = num_crash_cond_triggers / num_actions
    n_crashes = np.sum(np.array(df["action_made_by"]) == 0.0)-1
    last_timestamp = times[-1]
    crash_freq = n_crashes/last_timestamp
    avg_crash_probability = np.mean(np.array(df["crash_probabilities"]))
    avg_dob_dist = np.mean(np.abs(np.array(df["angular position"])))
    sd_angular_pos = np.std(np.array(df["angular position"]))
    avg_angvel_mag = np.mean(np.abs(np.array(df["angular velocity"])))
    sd_angvel = np.std(np.array(df["angular velocity"]))
    angvel_rms = np.sqrt(np.mean(np.array(df["angular velocity"])**2))
    avg_defl_mag = np.mean(np.abs(np.array(df["joystick deflection"])))
    frequency_data = len(times)/last_timestamp
    average_cont_non_zero_input = average_nonzero_subarray_size(np.array(df["joystick deflection"]))
    prop_anticipatory_deflections = np.sum(df['anticipatory_deflections']) / num_actions
    average_cont_zero_input = average_zero_subarray_size(np.array(df["joystick deflection"]))
    avg_dur_cont_non_zero_input = average_cont_non_zero_input/frequency_data
    avg_dur_cont_zero_input = average_cont_zero_input/frequency_data

    return pd.DataFrame( [{
        "trial_num": index,
        "isCatch": isCatch,
        "perc_destab_actions": destabilizing_prop*100,
        "perc_anticip_actions": prop_anticipatory_deflections * 100,
        "num_crashes": n_crashes,
        "average_crash_prob": avg_crash_probability,
        "crash_freqs": crash_freq,
        "mean_dist_dob": avg_dob_dist,
        "ang_pos_sd": sd_angular_pos,
        "vel_mag_mean": avg_angvel_mag,
        "ang_vel_sd": sd_angvel,
        "vel_rms": angvel_rms,
        "defl_mag_mean": avg_defl_mag,
        "avg_dur_cont_non_zero_inputs": avg_dur_cont_non_zero_input,
        "avg_dur_cont_zero_inputs": avg_dur_cont_zero_input,
        "perc_crash_cond_triggered": crash_cond_triggered_prop*100,
    }])



def plot_graphs(df, ppt_id, ppt_path, session, task, dp):

    # make position-velocity plots
    for i, x in enumerate([("other", "black"), ("destab", "red"), ("anticip", "blue"),]):
        label, color = x
        plt.scatter(df[df["deflection_type_label"] == i]["angular position"], df[df["deflection_type_label"] == i]["angular velocity"], marker=".", c=color, label=label, alpha=0.7)

    plt.xlabel("Angular Position (θ)")
    plt.ylabel("Angular Velocity (θ/s)")
    plt.xlim((-62, 62))
    plt.legend()
    plt.savefig(f"{'/'.join(dp.split('/')[:-1])}/position_velocity_plot.png", dpi=400)
    plt.close()
    


COLUMNS = ['task', 'perc_destab_actions', 'perc_anticip_actions', 'num_crashes',
       'average_crash_prob', 'crash_freqs', 'mean_dist_dob', 'ang_pos_sd',
       'vel_mag_mean', 'ang_vel_sd', 'vel_rms', 'defl_mag_mean',
       'avg_dur_cont_non_zero_inputs', 'avg_dur_cont_zero_inputs',
       'perc_crash_cond_triggered', ]

def process_data(ppt_id, ppt_path, session):
    print(f"processing {ppt_id}")

    # for each plot the position-velocity phase plots,
    # summarize performance for each task.
    participant_summaries = []
    for task in tasks[session]:
        print(f"    {task}")
        curr_path = f"{ppt_path}/{task}/"
        data_paths = glob.glob(f"{curr_path}*/*.csv")
        catch_trial = glob.glob(f"{curr_path}*/*.json")
        toSkip = set()
        if len(catch_trial):
            subpath = "/".join(catch_trial[0].split('/')[:-1])
            for p in data_paths:
                if subpath in p:
                    toSkip.add(p)
        # elif session=="1st_session" and task == "rdk_alone_AI_assist_first":
        #     toSkip.add(data_paths[0])

        task_summaries = []
        total_summary = {
            "trial_num": "summary",
            "isCatch": "summary",
            "perc_destab_actions": 0,
            "perc_anticip_actions": 0,
            "num_crashes": 0,
            "crash_freqs": 0,
            "mean_dist_dob": 0,
            "average_crash_prob": 0,
            "ang_pos_sd": 0,
            "vel_mag_mean": 0,
            "ang_vel_sd": 0,
            "vel_rms": 0,
            "defl_mag_mean": 0,
            "avg_dur_cont_non_zero_inputs": 0,
            "avg_dur_cont_zero_inputs": 0,
            "perc_crash_cond_triggered": 0,
        }
        for i, dp in enumerate(data_paths):
            df = pd.read_csv(dp)

            # adding more columns for certain features 
            check_anticipatory_deflection = lambda x: 1 if x[0]!=0 and np.sign(x[1])!=np.sign(x[2])  else 0
            df['anticipatory_deflections'] = df.apply(lambda x: check_anticipatory_deflection([x[1], x[2], x[3]]), axis=1)
            assign_label = lambda x,y: 0 if (x == y) or (not x and not y) else 1 if x == 1 else 2
            df['deflection_type_label'] = df.apply(lambda x: assign_label(x[7], x[10]), axis=1)
            
            plot_graphs(df, ppt_id, ppt_path, session, task, dp)
             
            summary = summarize_dataframe(df, i, 1 if dp in toSkip else 0)
            task_summaries.append(summary)
            if dp not in toSkip:
                for k in total_summary:
                    if k in ["trial_num", "isCatch"]:
                        continue
                    total_summary[k] += summary[k].iloc[0]
        if len(data_paths):
            for k in total_summary:
                if k in ["trial_num", "isCatch", "num_crashes"]:
                    continue
                total_summary[k] /= (len(data_paths) - len(toSkip))


            print(f"        {len(task_summaries)}")
            task_summaries.append(pd.DataFrame([total_summary]))
            summary_df = pd.concat(task_summaries)

            summary_df.to_csv(f"{ppt_path}/{task}_summary.csv", index=False)

            summary_df["task"] = summary_df["trial_num"].apply(lambda _: task)

            participant_summaries.append(summary_df[summary_df["trial_num"] == "summary"].drop(columns=["trial_num", "isCatch"]))
    
    if len(participant_summaries):
        print(f"    {len(participant_summaries)}")
        ppt_summary_df = pd.concat(participant_summaries)
        ppt_summary_df = ppt_summary_df[COLUMNS]
        
        ppt_summary_df.to_csv(f"{ppt_path}/{session}_summary.csv", index=False)


In [ ]:
for ppt_id, ppt_path in ppt_data_paths:
    process_data(ppt_id, ppt_path, "1st_session")
    

In [ ]:
for ppt_id, ppt_path in ppt_data_paths:
    process_data(ppt_id, ppt_path, "2nd_session")

In [ ]:
participant_model_mappings = pd.read_excel("../python_vip/participants_models_mappings.xlsx").head(20)
participant_model_mappings

In [ ]:
columns_shortHand = {
 'num_crashes': '# Crashes',
 'perc_destab_actions': '% destab.',
 'perc_anticip_actions': '% anticip.',
 'mean_dist_dob': 'μ|θ|',
 'ang_pos_sd': 'σ(θ)',
 'vel_mag_mean': 'μ|Mag|vel',
 'Angular velocity SD': 'σ(|Mag|vel)',
 'vel_rms': 'vel RMS',
 'Deflection magnitude mean': 'μ|d| * 30'
}


COLUMNS =  [ 'perc_destab_actions', 'perc_anticip_actions', 'num_crashes',
       'average_crash_prob', 'crash_freqs', 'mean_dist_dob', 'ang_pos_sd',
       'vel_mag_mean', 'ang_vel_sd', 'vel_rms', 'defl_mag_mean',
       'avg_dur_cont_non_zero_inputs', 'avg_dur_cont_zero_inputs',
        ]


assistant_map = {
    "A": "SAC",
    "B": "SAC-AIRL",
    "C": "DDPG",
    "D": "MLP",
    "E": "LSTM"
}

##### First session RDK ALone vs AI assistance

In [ ]:


COLUMNS_PLOT = ['num_crashes','perc_destab_actions', 'perc_destab_actions' , 'mean_dist_dob', 'ang_pos_sd', 'vel_mag_mean', 'vel_rms']

total_diff_data = []


for ppt_id, ppt_path in ppt_data_paths:
    model_used_sess_1 = participant_model_mappings[participant_model_mappings.participant == ppt_id]["model"].iloc[0]

    model_used_sess_2 = participant_model_mappings[participant_model_mappings.participant == ppt_id]["model_second_session"].iloc[0]
    print(ppt_id, model_used_sess_1)

    summary_1 = pd.read_csv(f"{ppt_path}/1st_session_summary.csv")


    for col in COLUMNS_PLOT:
        metric_diff = summary_1[summary_1.task == "rdk_alone_AI_assist_first"][col].values - summary_1[summary_1.task == "rdk_alone_first"][col].values
        
        metric_diff = metric_diff[0]
        if metric_diff == np.inf:
            metric_diff = 0
        
        
        total_diff_data.append(
            [ppt_id, assistant_map[model_used_sess_1], columns_shortHand[col], metric_diff] 
        )
    

total_diff_df1 = pd.DataFrame(total_diff_data, columns=["ppt", "model", "feature", "diff"])





##### Second session RDK ALone vs AI assistance original

In [ ]:


COLUMNS_PLOT = ['num_crashes', 'perc_destab_actions', 'mean_dist_dob', 'ang_pos_sd', 'vel_mag_mean', 'vel_rms']
total_diff_data = []


for ppt_id, ppt_path in ppt_data_paths[::-1]:
    model_used_sess_1 = participant_model_mappings[participant_model_mappings.participant == ppt_id]["model"].iloc[0]

    model_used_sess_2 = participant_model_mappings[participant_model_mappings.participant == ppt_id]["model_second_session"].iloc[0]
    print(ppt_id, model_used_sess_2)
    try:
        summary_1 = pd.read_csv(f"{ppt_path}/2nd_session_summary.csv")
        
        for col in COLUMNS_PLOT:
            metric_diff = summary_1[summary_1.task == "rdk_alone_AI_assist_second_original"][col].values - summary_1[summary_1.task == "rdk_alone_second"][col].values
            metric_diff = metric_diff[0]
            if metric_diff == np.inf:
                metric_diff = 0
            
            
            total_diff_data.append(
                [ppt_id, assistant_map[model_used_sess_2], columns_shortHand[col], metric_diff] 
            )
                
    except Exception as e:
        print(e)
    

total_diff_df2 = pd.DataFrame(total_diff_data, columns=["ppt", "model", "feature", "diff"])





##### Second session RDK ALone vs AI assistance retrained

In [ ]:


COLUMNS_PLOT = ['num_crashes', 'perc_destab_actions', 'mean_dist_dob', 'ang_pos_sd', 'vel_mag_mean', 'vel_rms']
total_diff_data = []


for ppt_id, ppt_path in ppt_data_paths[::-1]:
    model_used_sess_1 = participant_model_mappings[participant_model_mappings.participant == ppt_id]["model"].iloc[0]

    model_used_sess_2 = participant_model_mappings[participant_model_mappings.participant == ppt_id]["model_second_session"].iloc[0]
    print(ppt_id, model_used_sess_2)

    try:
        summary_1 = pd.read_csv(f"{ppt_path}/2nd_session_summary.csv")


        for col in COLUMNS_PLOT:
            metric_diff = summary_1[summary_1.task == "rdk_alone_AI_assist_second_retrained"][col].values - summary_1[summary_1.task == "rdk_alone_second"][col].values
            # print(metric_diff)
            metric_diff = metric_diff[0]
            if metric_diff == np.inf:
                metric_diff = 0
            
            
            total_diff_data.append(
                [ppt_id, assistant_map[model_used_sess_2], columns_shortHand[col], metric_diff] 
            )
                
    except Exception as e:
        print(e)
    

total_diff_df3 = pd.DataFrame(total_diff_data, columns=["ppt", "model", "feature", "diff"])





In [ ]:
import seaborn as sns
# sns.set(rc = {'figure.figsize':(1, 7),},)
sns.set(rc={'figure.figsize':(45,10)})
sns.set_theme(style="ticks", font_scale=2.5, palette="bright", context="paper" )

fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, sharey=True, sharex=True)

sns.boxplot(x="feature", y="diff", hue="model", data=total_diff_df1, ax=ax1)
ax1.set(xlabel="", ylabel='Absolute difference')
ax1.get_legend().remove()
# plt.legend(loc="upper left", bbox_to_anchor=(1, 0.5))

sns.boxplot(x="feature", y="diff", hue="model", data=total_diff_df2, ax=ax2)
# ax2.set(xlabel='Task metrics')
ax2.set_xlabel('Task metrics', labelpad=20)
ax2.get_legend().remove()

sns.boxplot(x="feature", y="diff", hue="model", data=total_diff_df3, ax=ax3)
ax3.set(xlabel='', ylabel='')
plt.legend(loc="upper left", bbox_to_anchor=(1, 0.7))

plt.ylim(-40, 60);

plt.savefig("./results/human_ai_study/combined_first_second-original_second-retrained_differences.pdf", dpi=300, bbox_inches="tight")
# sns.plt.show()

##### Second session alone difference vs first session 

In [ ]:
COLUMNS_PLOT = ['num_crashes', 'perc_destab_actions' , 'mean_dist_dob', 'ang_pos_sd', 'vel_mag_mean', 'vel_rms']
total_diff_data = []


for ppt_id, ppt_path in ppt_data_paths:
    model_used_sess_1 = participant_model_mappings[participant_model_mappings.participant == ppt_id]["model"].iloc[0]

    model_used_sess_2 = participant_model_mappings[participant_model_mappings.participant == ppt_id]["model_second_session"].iloc[0]
    print(ppt_id, model_used_sess_2)

    try:
        summary_1 = pd.read_csv(f"{ppt_path}/2nd_session_summary.csv")
        
        x = pd.read_csv(f"{ppt_path}/1st_session_summary.csv")
        for col in COLUMNS_PLOT:
            
            total_diff_data.append(
                [ppt_id, assistant_map[model_used_sess_1], columns_shortHand[col], x[x.task == "rdk_alone_first"][col].values[0], "1st session"] 
            )
            total_diff_data.append(
                [ppt_id, assistant_map[model_used_sess_1], columns_shortHand[col], summary_1[summary_1.task == "rdk_alone_second"][col].values[0], "2nd session"] 
            )
            
    except Exception as e:
        print(e)
    

total_diff_df = pd.DataFrame(total_diff_data, columns=["ppt", "model", "feature", "diff", "session"])





In [ ]:
import seaborn as sns
# sns.set(rc = {'figure.figsize':(1, 7),},)
sns.set_theme(style="ticks", font_scale=2.5, palette="bright", context="paper" )
# sns.set(rc={'figure.figsize':(11.7,8.27)})

# Load the example tips dataset
# tips = sns.load_dataset("tips")
# print(tips.head())
# Draw a nested boxplot to show bills by day and time
sns.boxplot(x="feature", y="diff", hue="session",
            data=total_diff_df, )

plt.xlabel("Task metrics", labelpad=20)
plt.ylabel("Average metric value over 3 trials");
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig(f"./results/human_ai_study/second_session_alone_difference_first_session.pdf", dpi=300, bbox_inches="tight");
# sns.despine(offset=10, trim=True)

Plot catch trial metrics for original and retrained models compared to last solo trial

In [ ]:
COLUMNS_PLOT = ['num_crashes', 'perc_destab_actions','mean_dist_dob', 'ang_pos_sd', 'vel_mag_mean', 'vel_rms']
total_diff_data = []


for ppt_id, ppt_path in ppt_data_paths[::-1]:
    print(ppt_id, )
    solo = pd.read_csv(f"{ppt_path}/rdk_alone_second_summary.csv")
    original = pd.read_csv(f"{ppt_path}/rdk_alone_AI_assist_second_original_summary.csv")
    retrained = pd.read_csv(f"{ppt_path}/rdk_alone_AI_assist_second_retrained_summary.csv")
    model_used_sess_2 = participant_model_mappings[participant_model_mappings.participant == ppt_id]["model_second_session"].iloc[0]
    for col in COLUMNS_PLOT:

        diff = original[original.isCatch == "1"][col].iloc[0] - solo[solo.trial_num == "2"][col].iloc[0]

        if diff == np.inf:
            diff = 0
        
        total_diff_data.append(
            [ppt_id, assistant_map[model_used_sess_2], columns_shortHand[col], diff, "original"]
        )

        diff = retrained[retrained.isCatch == "1"][col].iloc[0] - solo[solo.trial_num == "2"][col].iloc[0]

        if diff == np.inf:
            diff = 0
        total_diff_data.append(
            [ppt_id, assistant_map[model_used_sess_2], columns_shortHand[col], diff, "retrained"]
        )

total_diff_df = pd.DataFrame(total_diff_data, columns=["ppt", "model", "feature", "diff", "version"])





In [ ]:
import seaborn as sns
# sns.set(rc = {'figure.figsize':(1, 7),},)
sns.set_theme(style="ticks", font_scale=2.5, palette="bright", context="paper" )
# sns.set(rc={'figure.figsize':(11.7,8.27)})

# Load the example tips dataset
# tips = sns.load_dataset("tips")
# print(tips.head())
# Draw a nested boxplot to show bills by day and time
sns.boxplot(x="feature", y="diff",
            hue="model",
            data=total_diff_df[total_diff_df.version == "original"],
            )


plt.xlabel("Task metrics", labelpad=20)
plt.ylabel("Absolute difference");
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig(f"./results/human_ai_study/second_session_ai_assist_catch_trial_original.pdf", dpi=300, bbox_inches="tight");
# sns.despine(offset=10, trim=True)

In [ ]:
import seaborn as sns
# sns.set(rc = {'figure.figsize':(1, 7),},)
sns.set_theme(style="ticks", font_scale=2.5, palette="bright", context="paper" )
# sns.set(rc={'figure.figsize':(11.7,8.27)})

# Load the example tips dataset
# tips = sns.load_dataset("tips")
# print(tips.head())
# Draw a nested boxplot to show bills by day and time
sns.boxplot(x="feature", y="diff",
            hue="model",
            data=total_diff_df[total_diff_df.version == "retrained"],
            )

plt.xlabel("Task metrics", labelpad=20)
plt.ylabel("Absolute difference");
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig(f"./results/human_ai_study/second_session_ai_assist_catch_trial_retrained.pdf", dpi=300, bbox_inches="tight");
# sns.despine(offset=10, trim=True)

In [ ]:
def summarize_dataframe(df, index, isCatch):
    times = np.array(df['time'])
    num_actions = np.size(df["destabilizing_actions"])
    num_destabilizing_actions = np.sum(df["destabilizing_actions"])
    destabilizing_prop = num_destabilizing_actions/num_actions
    num_crash_cond_triggers = np.sum(df["is_crash_condition_triggered"])
    crash_cond_triggered_prop = num_crash_cond_triggers / num_actions
    n_crashes = np.sum(np.array(df["action_made_by"]) == 0.0)-1
    last_timestamp = times[-1]
    crash_freq = n_crashes/last_timestamp
    avg_crash_probability = np.mean(np.array(df["crash_probabilities"]))
    avg_dob_dist = np.mean(np.abs(np.array(df["angular position"])))
    sd_angular_pos = np.std(np.array(df["angular position"]))
    avg_angvel_mag = np.mean(np.abs(np.array(df["angular velocity"])))
    sd_angvel = np.std(np.array(df["angular velocity"]))
    angvel_rms = np.sqrt(np.mean(np.array(df["angular velocity"])**2))
    avg_defl_mag = np.mean(np.abs(np.array(df["joystick deflection"])))
    frequency_data = len(times)/last_timestamp
    average_cont_non_zero_input = average_nonzero_subarray_size(np.array(df["joystick deflection"]))
    prop_anticipatory_deflections = np.sum(df['anticipatory_deflections']) / num_actions
    average_cont_zero_input = average_zero_subarray_size(np.array(df["joystick deflection"]))
    avg_dur_cont_non_zero_input = average_cont_non_zero_input/frequency_data
    avg_dur_cont_zero_input = average_cont_zero_input/frequency_data

    return pd.DataFrame( [{
        "trial_num": index,
        "isCatch": isCatch,
        "perc_destab_actions": destabilizing_prop*100,
        "perc_anticip_actions": prop_anticipatory_deflections * 100,
        "num_crashes": n_crashes,
        "average_crash_prob": avg_crash_probability,
        "crash_freqs": crash_freq,
        "mean_dist_dob": avg_dob_dist,
        "ang_pos_sd": sd_angular_pos,
        "vel_mag_mean": avg_angvel_mag,
        "ang_vel_sd": sd_angvel,
        "vel_rms": angvel_rms,
        "defl_mag_mean": avg_defl_mag,
        "avg_dur_cont_non_zero_inputs": avg_dur_cont_non_zero_input,
        "avg_dur_cont_zero_inputs": avg_dur_cont_zero_input,
        "perc_crash_cond_triggered": crash_cond_triggered_prop*100,
    }])



def plot_graphs(df, model_name, model_path, session, task, dp):

    # make position-velocity plots
    for i, x in enumerate([("other", "black"), ("destab", "red"), ("anticip", "blue"),]):
        label, color = x
        plt.scatter(df[df["deflection_type_label"] == i]["angular position"], df[df["deflection_type_label"] == i]["angular velocity"], marker=".", c=color, label=label, alpha=0.7)

    plt.xlabel("Angular Position (θ)")
    plt.ylabel("Angular Velocity (θ/s)")
    plt.xlim((-62, 62))
    plt.legend()
    plt.savefig(f"{'/'.join(dp.split('/')[:-1])}/position_velocity_plot.png", dpi=400)
    plt.close()
    


COLUMNS = ['task', 'perc_destab_actions', 'perc_anticip_actions', 'num_crashes',
       'average_crash_prob', 'crash_freqs', 'mean_dist_dob', 'ang_pos_sd',
       'vel_mag_mean', 'ang_vel_sd', 'vel_rms', 'defl_mag_mean',
       'avg_dur_cont_non_zero_inputs', 'avg_dur_cont_zero_inputs',
       'perc_crash_cond_triggered', ]

def process_data(model_name, model_path, session):
    print(f"processing {model_name}")

    # for each plot the position-velocity phase plots,
    # summarize performance for each task.
    model_summaries = []
    for task in ["original", "retrained"]:
        print(f"    {task}")
        curr_path = f"{model_path}/{task}/"
        data_paths = glob.glob(f"{curr_path}*/*.csv")

        task_summaries = []
        total_summary = {
            "trial_num": "summary",
            "perc_destab_actions": 0,
            "perc_anticip_actions": 0,
            "num_crashes": 0,
            "crash_freqs": 0,
            "mean_dist_dob": 0,
            "average_crash_prob": 0,
            "ang_pos_sd": 0,
            "vel_mag_mean": 0,
            "ang_vel_sd": 0,
            "vel_rms": 0,
            "defl_mag_mean": 0,
            "avg_dur_cont_non_zero_inputs": 0,
            "avg_dur_cont_zero_inputs": 0,
            "perc_crash_cond_triggered": 0,
        }
        for i, dp in enumerate(data_paths):
            df = pd.read_csv(dp)

            # adding more columns for certain features 
            check_anticipatory_deflection = lambda x: 1 if x[0]!=0 and np.sign(x[1])!=np.sign(x[2])  else 0
            df['anticipatory_deflections'] = df.apply(lambda x: check_anticipatory_deflection([x[1], x[2], x[3]]), axis=1)
            assign_label = lambda x,y: 0 if (x == y) or (not x and not y) else 1 if x == 1 else 2
            df['deflection_type_label'] = df.apply(lambda x: assign_label(x[7], x[10]), axis=1)
            
            plot_graphs(df, model_name, model_path, session, task, dp)
             
            summary = summarize_dataframe(df, i, 0)
            task_summaries.append(summary)
            for k in total_summary:
                if k in ["trial_num", "isCatch"]:
                    continue
                total_summary[k] += summary[k].iloc[0]
        if len(data_paths):
            for k in total_summary:
                if k in ["trial_num", "isCatch", "num_crashes"]:
                    continue
                total_summary[k] /= len(data_paths)


            print(f"        {len(task_summaries)}")
            task_summaries.append(pd.DataFrame([total_summary]))
            summary_df = pd.concat(task_summaries)

            summary_df.to_csv(f"{model_path}/{task}_summary.csv", index=False)

            summary_df["task"] = summary_df["trial_num"].apply(lambda _: task)

            model_summaries.append(summary_df[summary_df["trial_num"] == "summary"].drop(columns=["trial_num", "isCatch"]))
    
    if len(model_summaries):
        print(f"    {len(model_summaries)}")
        model_summary_df = pd.concat(model_summaries)
        model_summary_df = model_summary_df[COLUMNS]
        
        model_summary_df.to_csv(f"{model_path}/{session}_summary.csv", index=False)



In [ ]:
root_path = "../output/ijcai_models/"
model_data_paths = glob.glob(f"{root_path}*")
model_data_paths.sort(key=lambda x: x.split("/")[-1].split("_")[-1])
model_names = [x.split('/')[-1].upper() for x in model_data_paths]
model_data_paths = list(zip(model_names, model_data_paths))

for model_name, model_dp in model_data_paths:
    print(model_name, model_dp)
    process_data(model_name, model_dp, "0")

In [ ]:

COLUMNS =  [ 'perc_destab_actions', 'perc_anticip_actions', 'num_crashes',
       'average_crash_prob', 'crash_freqs', 'mean_dist_dob', 'ang_pos_sd',
       'vel_mag_mean', 'ang_vel_sd', 'vel_rms', 'defl_mag_mean',
       'avg_dur_cont_non_zero_inputs', 'avg_dur_cont_zero_inputs',
        ]

COLUMNS_PLOT = ['num_crashes', 'perc_destab_actions','mean_dist_dob', 'ang_pos_sd', 'vel_mag_mean', 'vel_rms']
total_diff_data = []


for ppt_id, ppt_path in model_data_paths:
    print(ppt_id, ppt_path)
    
    original = pd.read_csv(f"{ppt_path}/original_summary.csv")
    retrained = pd.read_csv(f"{ppt_path}/retrained_summary.csv")
    
    for col in COLUMNS_PLOT:
        diff = retrained[retrained.trial_num == "summary"][col].iloc[0] - original[original.trial_num == "summary"][col].iloc[0]

        if diff == np.inf:
            diff = 0
        total_diff_data.append(
            [ppt_id, ppt_id, columns_shortHand[col], diff, "diff"]
        )

total_diff_df = pd.DataFrame(total_diff_data, columns=["ppt", "model", "feature", "diff", "version"])





In [ ]:
import seaborn as sns
# sns.set(rc = {'figure.figsize':(1, 7),},)
sns.set_theme(style="ticks", font_scale=2.5, palette="bright", context="paper" )
# sns.set(rc={'figure.figsize':(11.7,8.27)})

# Load the example tips dataset
# tips = sns.load_dataset("tips")
# print(tips.head())
# Draw a nested boxplot to show bills by day and time
sns.swarmplot(x="feature", y="diff",
            hue="model",
            data=total_diff_df, hue_order=[assistant_map[x] for x in assistant_map],
            size=10
            )

plt.xlabel("Task metrics", labelpad=20)
plt.ylabel("Absolute difference");
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig(f"./results/human_ai_study/difference_bw_original_retrained_models.pdf", dpi=300, bbox_inches="tight");

In [ ]:
import json 

human_study_data_path = "../output/human_ai_study/"

participants_models_mappings = pd.read_excel("../python_vip/participants_models_mappings.xlsx")

model_information = {}

with open("../python_vip/name_mappings.json") as f:
    model_information = json.load(f)

model_information

participant_data_paths = glob.glob(f"{human_study_data_path}*")
participant_ids = sorted([x.split('/')[-1] for x in participant_data_paths], 
                         key=lambda x: x.split('_')[-1])
# participant_ids

model_ppt_groups = {}

for model in model_information['models']:
    model_name = model_information['models'][model]['name']
    model_ppt_groups[model_name] = {}
    model_ppt_groups[model_name]['ppts']  = participants_models_mappings[participants_models_mappings['model'] == model].dropna()['participant'].to_list()[:4]

# model_ppt_groups

d = []

for model in model_ppt_groups:
    print(model)
    dis_episodes_per_model = []

    for x in model_ppt_groups[model]['ppts']:
        paths = glob.glob(f"{human_study_data_path}{x}/{tasks['1st_session'][3]}/*/disagreement_episodes.txt")
        # print(x, paths)
        t = []
        for p in paths:
            dis_episodes = np.loadtxt(p, delimiter=',')

            t.append(len(dis_episodes))
        d.append([model, x, np.average(t)])

d = pd.DataFrame(d, columns=["model", "ppt", "average"])


In [ ]:
d.groupby("model").describe().to_csv("./results/human_ai_study/HITL_disagreement_episodes_stats.csv")

In [ ]:
task_shorthand = {
    "rdk_alone_AI_assist_first": "RDK Assist 1", "rdk_alone_AI_assist_second_original": "RDK Assist 2 - Original", "rdk_alone_AI_assist_second_retrained": "RDK Assist 2 - Retrained"
}

In [ ]:
def num_helpful_suggestions(df: pd.DataFrame):

    count_followed = 0
    count_recovered = 0
    count_crashed = 0
    count_provided_suggestions = 0
    time_to_recover = []
    time_to_crash = []

    suggestion_started = False
    start_time = 0
    start_pos = 0
    curr_time = 0
    suggestion_direction = 0
    start_index = 0
    
    for i in range(len(df)):

        if not suggestion_started:
            suggestion_started = df.iloc[i]["action_made_by"] == 4 and abs(df.iloc[i]["angular position"]) > np.degrees(0.2)
            if suggestion_started:
                start_time = df.iloc[i]["time"]
                curr_time = start_time
                start_pos = df.iloc[i]["angular position"]
                start_index = i
                suggestion_direction = np.sign(df.iloc[i]["assistant_actions"])
        
        else:

            # check position: if returned to safe region, update counts elif still not safe, continue, if crashed update counts and continue
            
            if curr_time - df.iloc[i]["time"] <= 0.45:
                if suggestion_direction == np.sign(df.iloc[i]["pilot_actions"]):
                    count_followed += 1
            else:
                start_index += 1
                curr_time = df.iloc[start_index]["time"]
                suggestion_direction = np.sign(df.iloc[start_index]["assistant_actions"])

            
            
            pos = df.iloc[i]["angular position"]
            recovered = abs(pos) < np.degrees(0.2) and abs(pos) < abs(start_pos)
            crashed = abs(pos) >= 60
            if recovered:
                suggestion_started = False
                time_to_recover.append(df.iloc[i]["time"] - start_time)
                count_recovered += 1
            elif crashed :
                suggestion_started = False
                time_to_crash.append(df.iloc[i]["time"] - start_time)
                count_crashed += 1

            

        

    if len(time_to_crash) == 0:
        time_to_crash.append(0)
    if len(time_to_recover) == 0:
        time_to_recover.append(0)

    count_provided_suggestions = len(df[df["action_made_by"] == 4])
    return (
        count_recovered,
        np.average(time_to_recover),
        count_crashed,
        np.average(time_to_crash),
        count_followed,
        count_provided_suggestions
    )




In [ ]:
output1 = []
output1_catch = []
output2 = []
output2_catch = []
for ppt_id, ppt_path in ppt_data_paths:
    print(ppt_id, ppt_path)
    
    for task in ["rdk_alone_AI_assist_first", "rdk_alone_AI_assist_second_original", "rdk_alone_AI_assist_second_retrained"]:
        print(f"    {task}")
        curr_path = f"{ppt_path}/{task}/"
        data_paths = glob.glob(f"{curr_path}*/*.csv")
        catch_trial = glob.glob(f"{curr_path}*/*.json")
        toSkip = set()
        if len(catch_trial):
            subpath = "/".join(catch_trial[0].split('/')[:-1])
            for p in data_paths:
                if subpath in p:
                    toSkip.add(p)
        elif task == "rdk_alone_AI_assist_first":
            toSkip.add(data_paths[0])

        # print(len(toSkip))
        temp1 = []
        temp2 = []
        model_used = participant_model_mappings[participant_model_mappings.participant == ppt_id]["model" if task=="rdk_alone_AI_assist_first" else "model_second_session"].iloc[0]
        for i, dp in enumerate(data_paths):
            df = pd.read_csv(dp)
            count_recov, time_recov, count_crash, time_crash, count_followed, count_provided = num_helpful_suggestions(df)
            # print(count_followed)
            if dp not in toSkip:
                temp1.append(len(df[df.action_made_by == 4])/len(df))
                temp2.append([count_recov, time_recov, count_crash, time_crash, count_followed, len(df), count_provided])
            else:
                output1_catch.append([ppt_id, assistant_map[model_used], len(df[df.action_made_by == 4])/len(df), task_shorthand[task]])

                output2_catch.append([
                    ppt_id, assistant_map[model_used], task_shorthand[task], 
                    count_recov, time_recov, count_crash, time_crash, count_followed, len(df), count_provided
                ])

            # break
        # break
        temp2 = np.array(temp2)
        # print(temp2)
        output1.append([ppt_id, assistant_map[model_used], np.average(temp1), task_shorthand[task]])
        output2.append([ppt_id, assistant_map[model_used], task_shorthand[task],
                        np.sum(temp2[:, 0]), np.average(temp2[:, 1]), np.sum(temp2[:, 2]), np.average(temp2[:, 3]), np.sum(temp2[:, 4]), np.sum(temp2[:, 5]), np.sum(temp2[:, 6]),
                        ])

prop_suggestions_given = pd.DataFrame(output1, columns=["ppt", "model", "proportion", "task"])
prop_suggestions_given_catch = pd.DataFrame(output1_catch, columns=["ppt", "model", "proportion", "task"])

recovery_crash_stats = pd.DataFrame(output2, columns=["ppt", "model", "task", "count_recovered", "time_recovered", "count_crashed", "time_crashed", "count_followed", "length_trials", "count_provided_suggestions"])
recovery_crash_stats_catch = pd.DataFrame(output2_catch, columns=["ppt", "model", "task", "count_recovered", "time_recovered", "count_crashed", "time_crashed", "count_followed", "length_trials", "count_provided_suggestions"])


In [ ]:
recovery_crash_stats.to_csv("./results/human_ai_study/recovery_crash_stats.csv", index=False)
recovery_crash_stats_catch.to_csv("./results/human_ai_study/recovery_crash_stats_catch.csv", index=False)

In [ ]:
import seaborn as sns
# sns.set(rc = {'figure.figsize':(1, 7),},)
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_theme(style="ticks", font_scale=2.5, palette="bright", context="paper" )

# Load the example tips dataset
# tips = sns.load_dataset("tips")
# print(tips.head())
# Draw a nested boxplot to show bills by day and time
sns.boxplot(x="task", y="proportion",
            hue="model",
            data=prop_suggestions_given, fliersize=7)

plt.xlabel("Task", labelpad=20)
plt.ylabel("Proportion of trial");
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig(f"./results/human_ai_study/proportion_trial_given_suggestions.pdf", dpi=300, bbox_inches="tight");
# sns.despine(offset=10, trim=True)

In [ ]:
import seaborn as sns
# sns.set(rc = {'figure.figsize':(1, 7),},)
# sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_theme(style="ticks", font_scale=2.5, palette="bright", context="paper" )

# Load the example tips dataset
# tips = sns.load_dataset("tips")
# print(tips.head())
# Draw a nested boxplot to show bills by day and time
sns.boxplot(x="task", y="proportion",
            hue="model",
            data=prop_suggestions_given_catch, fliersize=7)

plt.xlabel("Task", labelpad=20)
plt.ylabel("Proportion of trial");
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig(f"./results/human_ai_study/proportion_trial_given_suggestions_catch.pdf", dpi=300, bbox_inches="tight");
# sns.despine(offset=10, trim=True)